In [ ]:
!pip install transformers jiwer
!pip install gtts
!pip install pydub

  Using cached jiwer-3.0.4-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.5 MB/s eta 0:00:00


In [ ]:

from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from pydub import AudioSegment
import numpy as np
from io import BytesIO

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

def convertaud(input_file_path):
    audio = AudioSegment.from_file(input_file_path)
    audio = audio.set_channels(1)
    audio = audio.set_frame_rate(16000)

    output_buffer = BytesIO()
    audio.export(output_buffer, format="wav")
    output_buffer.seek(0)
    return output_buffer

def aud2txt(audio_bytes):
    audio_bytes.seek(0)
    audio_segment = AudioSegment.from_file(audio_bytes)
    audio_array = np.array(audio_segment.get_array_of_samples())
    return pipe(audio_array)['text']

if __name__ == "__main__":
    input_audio_path = "/content/test.mp3"

    audio_bytes = convertaud(input_audio_path)
    transcription_text = aud2txt(audio_bytes)
    print("Transcription:", transcription_text)


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription:  Hi, hello, how are you?


In [ ]:
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, TrainingArguments, Trainer
import jiwer

dataset = load_dataset("/contents/dataset")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")

def preprocess_data(batch):
    batch["input_values"] = processor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_values[0]
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
dataset = dataset.map(preprocess_data, remove_columns=["audio", "text"])

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

training_args = TrainingArguments(
    output_dir="./whisper-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    num_train_epochs=3
)

def compute_metrics(pred):
    pred_str = processor.batch_decode(pred.predictions, skip_special_tokens=True)
    labels_ids = pred.label_ids
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
    wer = jiwer.wer(label_str, pred_str)
    return {"wer": wer}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./whisper-finetuned-final")


In [ ]:
import torch

model = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-final")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")

audio_input = {"audio": "/contents/tt.wav"}
audio = Audio(sampling_rate=16000)
audio_data = audio.decode_example(audio_input)
input_values = processor(audio_data["array"], sampling_rate=16000).input_values

with torch.no_grad():
    logits = model(input_values=torch.tensor([input_values])).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print("Transcription:", transcription)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `

Transcribed Text:  Hi, Hello, How are you? Good morning Hello Whisper, I am alright
Reference Text: hi hello how are you good morning hello whisper ai model
Word Error Rate (WER): 0.9090909090909091


In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import os

def text_to_wav(text, language, output_file_wav):
    try:
        tts = gTTS(text=text, lang=language, slow=False)
        temp_mp3 = f"{output_file_wav}.mp3"
        tts.save(temp_mp3)

        audio = AudioSegment.from_mp3(temp_mp3)
        audio.export(output_file_wav, format="wav")
        print(f"Audio saved successfully as {output_file_wav}")

        os.remove(temp_mp3)
    except Exception as e:
        print(f"Error: {str(e)}")


languages = {
    "english": {"text": "Hello, how are you today? I hope you have a great day.", "lang_code": "en"},
    "french": {"text": "Bonjour, comment ça va aujourd'hui ? J'espère que vous passerez une bonne journée.", "lang_code": "fr"},
    "spanish": {"text": "Hola, ¿cómo estás hoy? Espero que tengas un gran día.", "lang_code": "es"},
    "german": {"text": "Hallo, wie geht es dir heute? Ich hoffe, du hast einen schönen Tag.", "lang_code": "de"},
    "tamil": {"text": "வணக்கம், இன்று எப்படி இருக்கிறீர்கள்? உங்களுக்கு ஒரு நல்ல நாள் இருக்கும் என்று நம்புகிறேன்.", "lang_code": "ta"},
    "telugu": {"text": "హలో, మీరు ఈ రోజు ఎలా ఉన్నారు? మీకు మంచి రోజు ఉండాలని ఆశిస్తున్నాను.", "lang_code": "te"},
    "malayalam": {"text": "ഹലോ, ഇന്ന് എങ്ങനെ അനുഭവപ്പെടുന്നു? നല്ലൊരു ദിവസം ഉണ്ടാവട്ടെ എന്ന് പ്രതീക്ഷിക്കുന്നു.", "lang_code": "ml"},
    "hindi": {"text": "नमस्ते, आप आज कैसे हैं? आशा करता हूँ आपका दिन शुभ हो।", "lang_code": "hi"},
    "japanese": {"text": "こんにちは、今日はどうですか？素敵な一日をお過ごしください。", "lang_code": "ja"},
    "russian": {"text": "Привет, как ты сегодня? Надеюсь, у тебя будет отличный день.", "lang_code": "ru"}
}

for lang_name, lang_data in languages.items():
    text = lang_data["text"]
    lang_code = lang_data["lang_code"]
    output_file_wav = f"{lang_name}.wav"

    text_to_wav(text, lang_code, output_file_wav)


Audio saved successfully as english.wav
Audio saved successfully as french.wav
Audio saved successfully as spanish.wav
Audio saved successfully as german.wav
Audio saved successfully as tamil.wav
Audio saved successfully as telugu.wav
Audio saved successfully as malayalam.wav
Audio saved successfully as hindi.wav
Audio saved successfully as japanese.wav
Audio saved successfully as russian.wav


In [ ]:
from transformers import pipeline
import jiwer

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

languages_audio = {
    "english": {"audio_path": "/content/english.wav", "reference": "Hello, how are you today? I hope you have a great day."},
    "french": {"audio_path": "/content/french.wav", "reference": "Bonjour, comment ça va aujourd'hui ? J'espère que vous passerez une bonne journée."},
    "spanish": {"audio_path": "/content/spanish.wav", "reference": "Hola, ¿cómo estás hoy? Espero que tengas un gran día."},
    "german": {"audio_path": "/content/german.wav", "reference": "Hallo, wie geht es dir heute? Ich hoffe, du hast einen schönen Tag."},
    "tamil": {"audio_path": "/content/tamil.wav", "reference": "வணக்கம், இன்று எப்படி இருக்கிறீர்கள்? உங்களுக்கு ஒரு நல்ல நாள் இருக்கும் என்று நம்புகிறேன்."},
    "telugu": {"audio_path": "/content/telugu.wav", "reference": "హలో, మీరు ఈ రోజు ఎలా ఉన్నారు? మీకు మంచి రోజు ఉండాలని ఆశిస్తున్నాను."},
    "malayalam": {"audio_path": "/content/malayalam.wav", "reference": "ഹലോ, ഇന്ന് എങ്ങനെ അനുഭവപ്പെടുന്നു? നല്ലൊരു ദിവസം ഉണ്ടാവട്ടെ എന്ന് പ്രതീക്ഷിക്കുന്നു."},
    "hindi": {"audio_path": "/content/hindi.wav", "reference": "नमस्ते, आप आज कैसे हैं? आशा करता हूँ आपका दिन शुभ हो।"},
    "japanese": {"audio_path": "/content/japanese.wav", "reference": "こんにちは、今日はどうですか？素敵な一日をお過ごしください。"},
    "russian": {"audio_path": "/content/russian.wav", "reference": "Привет, как ты сегодня? Надеюсь, у тебя будет отличный день."}
}

print(f"{'Language':<12} {'WER':<10} {'Transcribed Text':<40} {'Reference Text':<40}")
print("-" * 120)

for lang_name, data in languages_audio.items():
    audio_path = data['audio_path']
    reference = data['reference']

    try:
        result = pipe(audio_path)
        transcription = result['text']

        wer = jiwer.wer(reference, transcription)

        print(f"{lang_name.capitalize():<12} {wer:<10.4f} {transcription:<40} {reference:<40}")

    except Exception as e:
        print(f"Error processing {lang_name}: {str(e)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

Hindi        0.1667      नमस्ते, आप आज कैसे हैं? आशा करता हूँ आपका दिन शुब हो. नमस्ते, आप आज कैसे हैं? आशा करता हूँ आपका दिन शुभ हो।


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Japanese     1.0000     こんにちは今日はどうですか素敵な1日をお過ごしください              こんにちは、今日はどうですか？素敵な一日をお過ごしください。          


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Russian      0.0000      Привет, как ты сегодня? Надеюсь, у тебя будет отличный день. Привет, как ты сегодня? Надеюсь, у тебя будет отличный день.


In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import os

def text_to_wav(text, language, output_file_wav):
    try:
        tts = gTTS(text=text, lang=language, slow=False)
        temp_mp3 = f"{output_file_wav}.mp3"
        tts.save(temp_mp3)

        audio = AudioSegment.from_mp3(temp_mp3)
        audio.export(output_file_wav, format="wav")
        print(f"Audio saved successfully as {output_file_wav}")

        os.remove(temp_mp3)
    except Exception as e:
        print(f"Error: {str(e)}")


languages = {
    "english": {"text": "Hello, how are you today? I hope you have a great day.", "lang_code": "en"},
    "french": {"text": "Bonjour, comment ça va aujourd'hui ? J'espère que vous passerez une bonne journée.", "lang_code": "fr"},
    "spanish": {"text": "Hola, ¿cómo estás hoy? Espero que tengas un gran día.", "lang_code": "es"},
    "german": {"text": "Hallo, wie geht es dir heute? Ich hoffe, du hast einen schönen Tag.", "lang_code": "de"},
    "tamil": {"text": "Vanakkam, inru eppadi irukkinga? Ungalukku oru nalla naal irukkum enru nambugiren.", "lang_code": "ta"},
    "telugu": {"text": "Halo, meeru ee roju ela unnaru? Meeku manchi roju undaalani aashistunnaanu.", "lang_code": "te"},
    "malayalam": {"text": "Halo, innu engane anu anubhavappedunnathu? Nalloru divasam undavatte ennu pratheekshikkunnu.", "lang_code": "ml"},
    "hindi": {"text": "Namaste, aap aaj kaise hain? Aasha karta hoon aapka din shubh ho.", "lang_code": "hi"},
    "japanese": {"text": "Konnichiwa, kyou wa dou desu ka? Suteki na ichinichi wo sugoshite kudasai.", "lang_code": "ja"},
    "russian": {"text": "Privet, kak ty segodnya? Nadeyus', u tebya budet otlichnyy den'.", "lang_code": "ru"}
}

for lang_name, lang_data in languages.items():
    text = lang_data["text"]
    lang_code = lang_data["lang_code"]
    output_file_wav = f"{lang_name}.wav"

    text_to_wav(text, lang_code, output_file_wav)


Audio saved successfully as english.wav
Audio saved successfully as french.wav
Audio saved successfully as spanish.wav
Audio saved successfully as german.wav
Audio saved successfully as tamil.wav
Audio saved successfully as telugu.wav
Audio saved successfully as malayalam.wav
Audio saved successfully as hindi.wav
Audio saved successfully as japanese.wav
Audio saved successfully as russian.wav


In [ ]:
from transformers import pipeline
import jiwer

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

languages_audio = {
    "english": {"audio_path": "/content/english.wav", "reference": "Hello, how are you today? I hope you have a great day."},
    "french": {"audio_path": "/content/french.wav", "reference": "Bonjour, comment ça va aujourd'hui ? J'espère que vous passerez une bonne journée."},
    "spanish": {"audio_path": "/content/spanish.wav", "reference": "Hola, ¿cómo estás hoy? Espero que tengas un gran día."},
    "german": {"audio_path": "/content/german.wav", "reference": "Hallo, wie geht es dir heute? Ich hoffe, du hast einen schönen Tag."},
    "tamil": {"audio_path": "/content/tamil.wav", "reference": "Vanakkam, inru eppadi irukkinga? Ungalukku oru nalla naal irukkum enru nambugiren."},
    "telugu": {"audio_path": "/content/telugu.wav", "reference": "Halo, meeru ee roju ela unnaru? Meeku manchi roju undaalani aashistunnaanu."},
    "malayalam": {"audio_path": "/content/malayalam.wav", "reference": "Halo, innu engane anu anubhavappedunnathu? Nalloru divasam undavatte ennu pratheekshikkunnu."},
    "hindi": {"audio_path": "/content/hindi.wav", "reference": "Namaste, aap aaj kaise hain? Aasha karta hoon aapka din shubh ho."},
    "japanese": {"audio_path": "/content/japanese.wav", "reference": "Konnichiwa, kyou wa dou desu ka? Suteki na ichinichi wo sugoshite kudasai."},
    "russian": {"audio_path": "/content/russian.wav", "reference": "Privet, kak ty segodnya? Nadeyus', u tebya budet otlichnyy den'."}
}

print(f"{'Language':<12} {'WER':<10} {'Transcribed Text':<40} {'Reference Text':<40}")
print("-" * 120)

for lang_name, data in languages_audio.items():
    audio_path = data['audio_path']
    reference = data['reference']

    try:
        result = pipe(audio_path)
        transcription = result['text']

        wer = jiwer.wer(reference, transcription)

        print(f"{lang_name.capitalize():<12} {wer:<10.4f} {transcription:<40} {reference:<40}")

    except Exception as e:
        print(f"Error processing {lang_name}: {str(e)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Language     WER        Transcribed Text                         Reference Text                          
------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `

English      0.0000      Hello, how are you today? I hope you have a great day. Hello, how are you today? I hope you have a great day.


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


...

Hindi        1.0000      नमस्ते, आप आज कैसे हैं? आशा करता हूँ आपका दिन शुब हो. Namaste, aap aaj kaise hain? Aasha karta hoon aapka din shubh ho.


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Japanese     1.0000     こんにちは今日はどうですか素敵な1日を過ごしてください              Konnichiwa, kyou wa dou desu ka? Suteki na ichinichi wo sugoshite kudasai.


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Russian      1.0000      Привет, как тебе сегодня? Надеюсь, у тебя пойдет отличный день. Privet, kak ty segodnya? Nadeyus', u tebya budet otlichnyy den'.


In [ ]:
!pip install torch torchvision torchaudio openai-whisper tensor2tensor tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dopamine-rl to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dopamine-rl to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of kfac to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.4/981.4 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5

In [ ]:
!pip install torch torchvision torchaudio openai-whisper tensor2tensor==1.15.0 tensorflow==1.15.0



ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2)
ERROR: No matching distribution found for tensorflow==1.15.0


In [ ]:
import whisper
import numpy as np
from tensor2tensor import problems
from tensor2tensor.utils import registry
import tensorflow as tf

# Load Whisper model for voice recognition
whisper_model = whisper.load_model("base")

def transcribe_audio(audio_file):
    try:
        # Load audio and transcribe
        result = whisper_model.transcribe(audio_file)
        return result['text']
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

def translate_text(text):
    try:
        # Register the translation problem for English to German
        problem = problems.problem("translate_ende_wmt32k")

        # Prepare input for the translation problem
        encoded_text = problem.encode(text)

        # Decode to get the translated text
        translated_output = problem.decode(encoded_text)
        return translated_output
    except Exception as e:
        print(f"Error translating text: {e}")
        return None

def main(audio_file):
    # Transcribe the audio to text
    transcribed_text = transcribe_audio(audio_file)
    if transcribed_text:
        print("Transcribed Text:", transcribed_text)

        # Translate the transcribed text to German
        translated_text = translate_text(transcribed_text)
        if translated_text:
            print("Translated Text:", translated_text)

if __name__ == "__main__":
    audio_file = "/content/tamiltest.wav"  # Replace with your audio file path
    main(audio_file)


AttributeError: module 'tensorflow.compat.v1' has no attribute 'estimator'

In [ ]:
import whisper
from transformers import MarianMTModel, MarianTokenizer

# Load Whisper model for voice recognition
whisper_model = whisper.load_model("base")

def transcribe_audio(audio_file):
    try:
        # Load audio and transcribe
        result = whisper_model.transcribe(audio_file)
        return result['text']
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

def translate_text(text, target_language="en"):
    try:
        # Load the MarianMT model and tokenizer for translation to English
        model_name = f"Helsinki-NLP/opus-mt-{target_language}-en"
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        # Prepare input for the translation model
        encoded_text = tokenizer(text, return_tensors="pt", padding=True)

        # Translate and decode
        translated_output = model.generate(**encoded_text)
        translated_text = tokenizer.decode(translated_output[0], skip_special_tokens=True)

        return translated_text
    except Exception as e:
        print(f"Error translating text: {e}")
        return None

def main(audio_file):
    # Transcribe the audio to text
    transcribed_text = transcribe_audio(audio_file)
    if transcribed_text:
        print("Transcribed Text:", transcribed_text)

        # Specify the source language code (e.g., 'fr' for French, 'de' for German, etc.)
        source_language = "fr"  # Replace with the appropriate language code
        translated_text = translate_text(transcribed_text, target_language=source_language)
        if translated_text:
            print("Translated Text (to English):", translated_text)

if __name__ == "__main__":
    audio_file = "/content/tamiltest.wav"  # Replace with your audio file path
    main(audio_file)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 75.7MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribed Text:  போன்று படம்


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translated Text (to English): ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,  ,,,,,,,, ,,


In [ ]:
!pip install diffusers transformers accelerate scipy
!pip install torch


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

text_prompt = "A futuristic cityscape at sunset, with flying cars and neon lights"

image = pipe(text_prompt).images[0]

image.show()

image.save("generated_image.png")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
from diffusers import LDMPipeline
import torch
from PIL import Image


model_id = "CompVis/ldm-text2im-large-256"
pipe = LDMPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

text_prompt = "Dog waking on a futuritic city on streets"

image = pipe(text_prompt).images[0]

image.show()
image.save("generated_image_ldm.png")


model_index.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors not found


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

vqvae/config.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--ldm-text2im-large-256/snapshots/30de525ca11a880baea4962827fb6cb0bb268955/vqvae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--ldm-text2im-large-256/snapshots/30de525ca11a880baea4962827fb6cb0bb268955/vqvae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'timestep_values': None} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--ldm-text2im-large-256/snapshots/30de525ca11a880baea4962827fb6cb0bb268955/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--ldm-text2im-large-256/snapshots/30de525ca11a880baea4962827fb6cb0bb268955/unet

TypeError: randn() received an invalid combination of arguments - got (tuple, layout=torch.layout, dtype=NoneType, device=NoneType, generator=NoneType), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
